This notebook is a working space for surrogate model workflow modifications

In [3]:
import pandas as pd
def get_data(
    tool="pysmo",
    input_data_file="./results_100/input_data.csv",
    output_data_file="./results_100/output_data.csv",
):
    input_data = pd.read_csv(input_data_file, header=0)
    # print(input_data)
    output_data = pd.read_csv(output_data_file, header=0).iloc[:, 1:]
    # print(output_data)
    feed_data = pd.concat([input_data, output_data], axis=1)
    return feed_data, input_data, output_data

In [4]:
feed_data, input_data, output_data = get_data()
output_data.head(5)

,S_su,S_aa,S_fa,S_va,S_bu,S_pro,S_ac,S_h2,S_ch4,S_IC,...,bge2_Volumetric Flowrate,bgr2_ S_h2,bgr2_ S_ch4,bgr2_ S_IC,bgr2_ H2O,bgr2_Volumetric Flowrate,bge1_ S_h2,bge1_ S_ch4,bge1_ S_IC,bge1_Volumetric Flowrate
0,2.020680,1.667568,27.803600,3.183734,4.090462,5.092324,19.929851,0.000022,47.364232,554.701763,...,1.0,0.002782,2571.570151,98.499511,32.250790,12816.938977,3730.785472,1.043148e+07,4.543148e+06,1.0
1,0.698211,0.723826,20.154450,1.193648,1.513911,1.653812,13.042476,0.000009,46.802556,577.751962,...,1.0,0.001136,2466.439439,96.671460,25.130551,9135.159753,1089.106113,7.943233e+06,3.104289e+06,1.0
2,1.116320,1.061000,23.629892,1.911950,2.422099,3.183216,15.194866,0.000014,54.972807,583.874219,...,1.0,0.001777,2703.484283,92.703332,20.334959,12209.350415,2447.598726,1.123190e+07,4.584090e+06,1.0
3,0.677144,0.705968,19.939071,1.155851,1.466414,1.585702,12.844674,0.000008,52.960528,593.155806,...,1.0,0.001102,2728.626555,96.999883,22.878091,13755.114637,1766.922909,1.329722e+07,5.184483e+06,1.0
4,1.179762,1.108334,24.033173,2.012124,2.549928,3.404492,16.242747,0.000015,42.388289,553.280864,...,1.0,0.001894,2440.133280,101.727901,37.048089,11424.063724,2150.378021,9.352876e+06,3.839280e+06,1.0


In [5]:
input_labels = list(input_data.columns)
output_labels = [list(output_data.columns)[0]]
output_labels

['S_su']

In [6]:
feed_data_cut = feed_data[list(input_data.columns)+output_labels]
feed_data_cut.head(5)

,inf_fr,temp,hrt,S_su
0,7445.41,31.36,4.22,2.020680
1,5090.40,26.80,10.46,0.698211
2,7521.30,22.83,6.73,1.116320
3,8500.65,25.01,10.79,0.677144
4,6298.69,34.16,6.43,1.179762


In [ ]:
from idaes.core.surrogate.sampling.data_utils import split_training_validation
from idaes.core.surrogate.pysmo_surrogate import PysmoPolyTrainer, PysmoSurrogate
from idaes.core.surrogate.plotting.sm_plotter import (
    surrogate_scatter2D,
    surrogate_parity,
    surrogate_residual,
)

xmin, xmax = input_data.min().tolist(), input_data.max().tolist()
input_bounds = {
    input_labels[i]: (xmin[i], xmax[i]) for i in range(len(input_labels))
}
# Create PySMO trainer object
trainer = PysmoPolyTrainer(
    input_labels=input_labels,
    output_labels=output_labels,
    training_dataframe=feed_data_cut,
)

# Set PySMO options
trainer.config.maximum_polynomial_order = 6
trainer.config.multinomials = True
trainer.config.training_split = 0.8
trainer.config.number_of_crossvalidations = 3

from idaes.core.surrogate.pysmo_surrogate import PysmoSurrogateTrainingResult
model = trainer._create_model(feed_data_cut, output_labels)
model.training()
performacetrainer._get_metrics(model)



===========================Polynomial Regression===============================================


No iterations will be run.
Default parameter estimation method is used.
Parameter estimation method:  pyomo 

max_fraction_training_samples set at  0.5
Number of adaptive samples (no_adaptive_samples) set at  4
Maximum number of iterations (Max_iter) set at:  0
model.name="unknown";
    - termination condition: maxIterations
    - message from solver: Ipopt 3.13.2\x3a Maximum Number of Iterations
      Exceeded.
model.name="unknown";
    - termination condition: maxIterations
    - message from solver: Ipopt 3.13.2\x3a Maximum Number of Iterations
      Exceeded.
model.name="unknown";
    - termination condition: maxIterations
    - message from solver: Ipopt 3.13.2\x3a Maximum Number of Iterations
      Exceeded.
model.name="unknown";
    - termination condition: maxIterations
    - message from solver: Ipopt 3.13.2\x3a Maximum Number of Iterations
      Exceeded.
model.name="unknown";
 

{'RMSE': 0.12867944982293417, 'R2': 0.9977383091273306}

In [ ]:
# Train surrogate (calls PySMO through IDAES Python wrapper)
poly_train = trainer.train_surrogate()
poly_surr = PysmoSurrogate(
            poly_train, input_labels, output_labels, input_bounds
        )

In [56]:
method_list= []
method = 'alamo'#"rbf"#"kri"'poly'
path = "./results/"
file = path + method + "_surrogate.json"
import json

# Assuming 'data.json' contains: {"product": "Laptop", "price": 1200}
with open(file, 'r') as file:
    data = json.load(file)

#print(data['model_encoding']['S_su']['attr']["training_R2"])

#print(data['model_encoding']['S_su']['attr']['errors'])
if method == 'poly':
    metrics_sum = pd.DataFrame()
    for ele in data['model_encoding']:
        metrics = data['model_encoding'][ele]['attr']['errors']
        metrics["Comp"] = ele
        #print(metrics)
        for key in metrics:
            metrics[key] = [metrics[key]]
        #print(metrics)
        df = pd.DataFrame.from_dict(metrics)
        metrics_sum = pd.concat([metrics_sum, df])



elif method == 'kri':
    metrics_sum = pd.DataFrame()
    for ele in data['model_encoding']:
        metrics = {}
        metrics['R2'] = [data['model_encoding'][ele]['attr']["training_R2"]]
        metrics['RMSE'] = [data['model_encoding'][ele]['attr']["training_rmse"]]
        metrics["Comp"] = [ele]
        df = pd.DataFrame.from_dict(metrics)
        metrics_sum = pd.concat([metrics_sum, df])

    #print(data['model_encoding']['S_su']['attr']["training_R2"])
    #print(data['model_encoding']['S_su']['attr']["training_rmse"])


elif method == "rbf":
    metrics_sum = pd.DataFrame()
    for ele in data['model_encoding']:
        metrics = {}
        metrics['R2'] = [data['model_encoding'][ele]['attr']["R2"]]
        metrics['RMSE'] = [data['model_encoding'][ele]['attr']["rmse"]]
        metrics["Comp"] = [ele]
        df = pd.DataFrame.from_dict(metrics)
        metrics_sum = pd.concat([metrics_sum, df])

elif method == "alamo":
    metrics_sum = pd.DataFrame()
    print(data['surrogate']['S_su'])#['model_encoding']['S_su']['attr']["training_R2"])


# metrics_sum_indexed = metrics_sum.set_index('Comp')   
# print(metrics_sum_indexed)
# metrics_sum_indexed.to_csv(path + method + '_metircs.csv')





 S_su == 43.989350626853934045357 * inf_fr - 9.4706210126369274604485 * temp + 19.893669255685551888746 * hrt + 114.59817145755525302775 * ln(inf_fr) + 129.01061341366997226032 * ln(temp) - 37.008209572391699282434 * ln(hrt) - 0.56234520123311817867140E-003 * exp(inf_fr) - 0.15090473026542536424868E-004 * exp(hrt) + 3.0534771747869551283827 * sin(inf_fr) + 0.53360116351520965952138E-002 * sin(temp) + 0.20119079821944874097994 * sin(hrt) - 2.8232394802081119955517 * cos(inf_fr) + 0.89481058687089873315834E-001 * cos(temp) - 0.49695539002178712451041 * cos(hrt) - 10.636553297318570798780 * inf_fr**2 + 0.85847680887134125704563E-001 * temp**2 - 1.7623608887704267367269 * hrt**2 + 0.57907763786762767033167 * inf_fr**3 + 0.65304994980003444604577E-001 * hrt**3 - 441.75496539900063908135
